<a href="https://colab.research.google.com/github/mille055/AIPI540-Deep-Learning-Applications/blob/main/3_nlp/classification/text_classification_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# Text Classification using Word Counts / TFIDF
In this notebook we will be performing text classification by using word counts and frequency to create numerical feature vectors representing each text document and then using these features to train a simple classifier.  Although simple, we will see that this approach can work very well for classifying text, even compared to more modern document embedding approaches.  Our goal will be to classify the articles in the AgNews dataset into their correct category: "World", "Sports", "Business", or "Sci/Tec".

**Notes:**  
- This does not need to be run on GPU, but will take ~5 minutes to run  



In [7]:
import os
import numpy as np
import pandas as pd
import string
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
import urllib.request
import zipfile

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
!python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_sm')

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-05 13:07:44.201653: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 13:07:44.201832: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 13:07:44.201866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Download and prepare data

In [8]:
# Download the data
if not os.path.exists('../data'):
    os.mkdir('../data')
if not os.path.exists('../data/agnews'):
    url = 'https://storage.googleapis.com/aipi540-datasets/agnews.zip'
    urllib.request.urlretrieve(url,filename='../data/agnews.zip')
    zip_ref = zipfile.ZipFile('../data/agnews.zip', 'r')
    zip_ref.extractall('../data/agnews')
    zip_ref.close()

train_df = pd.read_csv('../data/agnews/train.csv')
test_df = pd.read_csv('../data/agnews/test.csv')

# Combine title and description of article to use as input documents for model
train_df['full_text'] = train_df.apply(lambda x: ' '.join([x['Title'],x['Description']]),axis=1)
test_df['full_text'] = test_df.apply(lambda x: ' '.join([x['Title'],x['Description']]),axis=1)

# Create dictionary to store mapping of labels
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

train_df.head()

,Class Index,Title,Description,full_text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


In [9]:
# View a couple of the documents
for i in range(5):
    print(train_df.iloc[i]['full_text'], i)
    print()

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again. 0

Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market. 1

Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums. 2

Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday. 3

Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaw

## Pre-process text
Before we create our features, we first need to pre-process our text.  There are several methods to pre-process text; in this example we will perform the following operations on our raw text to prepare it for creating features:  
- Tokenize our raw text to break it into a list of substrings.  This step primarily splits our text on white space and punctuation.  As an example from the [NLTK](https://www.nltk.org/api/nltk.tokenize.html) website:  

    ```
    >>> s = "Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks."
    >>> word_tokenize(s)
    ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.',
    'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']
    ```

- Remove punctuation and stopwords.  Stopwords are extremely commonly used words (e.g. "a", "and", "are", "be", "from" ...) that do not provide any useful information to us to assist in modeling the text.  

- Lemmatize the words in each document.  Lemmatization uses a morphological analysis of words to remove inflectional endings and return the base or dictionary form of words, called the "lemma".  Among other things, this helps by replacing plurals with singular form e.g. "dogs" becomes "dog" and "geese" becomes "goose".  This is particularly important when we are using word counts or freqency because we want to count the occurences of "dog" and "dogs" as the same word.  

There are several libraries available in Python to process text.  Below we have shown how to perform the above operations using two of the most popular: [NLTK](https://www.nltk.org) and [Spacy](https://spacy.io).

In [43]:
def tokenize(sentence,method='spacy'):
# Tokenize and lemmatize text, remove stopwords and punctuation

    punctuations = string.punctuation
    stopwords = list(STOP_WORDS)

    sentence = sentence.replace("\\"," ")
    
    if method=='nltk':
        # Tokenize
        tokens = nltk.word_tokenize(sentence,preserve_line=True)
        # Remove stopwords and punctuation
        
        tokens = [word for word in tokens if word not in stopwords and word not in punctuations]
        
        # Lemmatize
        wordnet_lemmatizer = WordNetLemmatizer()
        tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
        tokens = " ".join([i for i in tokens])
    else:
        # Tokenize
        with nlp.select_pipes(enable=['tokenizer','lemmatizer']):
            tokens = nlp(sentence)
        # Lemmatize
        tokens = [word.lemma_.lower().strip() for word in tokens]
        # Remove stopwords and punctuation
        tokens = [word for word in tokens if word not in stopwords and word not in punctuations]
        tokens = " ".join([i for i in tokens])
    return tokens

In [51]:
# Process the training set text
tqdm.pandas()
train_df['processed_text'] = train_df['full_text'].progress_apply(lambda x: tokenize(x,method='nltk'))

# Process the test set text
tqdm.pandas()
test_df['processed_text'] = test_df['full_text'].progress_apply(lambda x: tokenize(x,method='nltk'))

100%|██████████| 7600/7600 [00:04<00:00, 1655.01it/s]


In [52]:
## show the difference between the full text and the processed text
display(train_df.iloc[1].processed_text)
display(train_df.iloc[1].full_text)

'Carlyle Looks Toward Commercial Aerospace Reuters Reuters Private investment firm Carlyle Group reputation making well-timed occasionally controversial play defense industry quietly placed bet market'

'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'

## Create features using word counts
Now that our raw text is pre-processed, we are ready to create our features.  There are two approaches to creating features using word counts: **Count Vectorization** and **TFIDF Vectorization**.

**Count Vectorization** (also called Bag-of-words) creates a vocabulary of all words appearing in the training corpus, and then for each document it counts up how many times each word in the vocabulary appears in the document.  Each document is then represented by a vector with the same length as the vocabulary.  At each index position an integer indicates how many times each word appears in the document.

**Term Frequency Inverse Document Frequency (TFIDF) Vectorization** first counts the number of times each word appears in a document (similar to Count Vectorization) but then divides by the total number of words in the document to calculate the *term frequency (TF)* of each word.  The *inverse document frequency (IDF)* for each word is then calculated as the log of the total number of documents divided by the number of documents containing the word.  The TFIDF for each word is then computed by multiplying the term frequency by the inverse document frequency.  Each document is represented by a vector containing the TFIDF for every word in the vocabulary, for that document.

In the below `build_features()` function, you can specify whether to create document features using Count Vectorization or TFIDF Vectorization.

In [53]:
def build_features(train_data, test_data, ngram_range, method='count'):
    if method == 'tfidf':
        # Create features using TFIDF
        vec = TfidfVectorizer(ngram_range=ngram_range)
        X_train = vec.fit_transform(train_df['processed_text'])
        X_test = vec.transform(test_df['processed_text'])

    else:
        # Create features using word counts
        vec = CountVectorizer(ngram_range=ngram_range)
        X_train = vec.fit_transform(train_df['processed_text'])
        X_test = vec.transform(test_df['processed_text'])

    return X_train, X_test

In [54]:
# Create features
method = 'tfidf'
ngram_range = (1, 2)
X_train,X_test = build_features(train_df['processed_text'],test_df['processed_text'],ngram_range,method)

## Train model
Now that we have created our features representing each document, we will use them in a simple softmax regression classification model to predict the document's class.  We first train the classification model on the training set.

In [55]:
# Train a classification model using logistic regression classifier
y_train = train_df['Class Index']
logreg_model = LogisticRegression(solver='saga')
logreg_model.fit(X_train,y_train)
preds = logreg_model.predict(X_train)
acc = sum(preds==y_train)/len(y_train)
print('Accuracy on the training set is {:.3f}'.format(acc))

Accuracy on the training set is 0.961


## Evaluate model
We then evaluate our model on the test set.  As you can see, the model performs very well on this task, using this simple approach!  In general, Count Vectorization / TFIDF Vectorization performs surprising well across a broad range of tasks, even compared to more computationally intensive approaches such as document embeddings.  This should perhaps not be surprising, since we would expect documents about similar topics to contain similar sets of words.

In [56]:
# Evaluate accuracy on the test set
y_test = test_df['Class Index']
test_preds = logreg_model.predict(X_test)
test_acc = sum(test_preds==y_test)/len(y_test)
print('Accuracy on the test set is {:.3f}'.format(test_acc))

Accuracy on the test set is 0.919
